In [1]:
import pandas as pd
import numpy as np
from PIL import Image 
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import cv2

In [2]:
Thickness = pd.read_csv('./data/Thickness.csv')
Thickness = Thickness.drop([33]) # 9-1 drop
Thickness.reset_index(drop=True, inplace=True)
Thickness

,Thickness
0,48.266667
1,47.966667
2,48.166667
3,48.600000
4,59.533333
...,...
122,84.633333
123,98.366667
124,97.033333
125,96.266667


In [3]:
def hist_match(source, template):
    # 소스 이미지와 템플릿 이미지의 히스토그램과 누적 분포를 계산
    source_hist, _ = np.histogram(source.flatten(), bins=256, range=[0,256])
    template_hist, _ = np.histogram(template.flatten(), bins=256, range=[0,256])
    source_cumulative_hist = source_hist.cumsum()
    template_cumulative_hist = template_hist.cumsum()

    # # 히스토그램 정규화
    # source_cumulative_hist = (source_cumulative_hist - source_cumulative_hist.min()) * 255 / (source_cumulative_hist.max() - source_cumulative_hist.min())
    # template_cumulative_hist = (template_cumulative_hist - template_cumulative_hist.min()) * 255 / (template_cumulative_hist.max() - template_cumulative_hist.min())

    # 히스토그램 일치화 수행
    hist_match = np.interp(source_cumulative_hist, template_cumulative_hist, np.arange(256))

    print(type(hist_match))
    # 일치화 매핑을 소스 이미지에 적용
    matched_image = hist_match[source]
    print(type(matched_image))
    return matched_image.reshape(source.shape).astype(np.uint8)

In [4]:
img_path_list = []
Top_path = './data/box_img/Top/'
Bot_path = './data/box_img/Bot/'

for i in range(1, 33):
    for j in range(1, 5):
        if i == 9 and j == 1: 
            continue
        img_path_list.append('./data/test_img/AI-{}_0{}_transformed.jpg'.format(i, j))

In [7]:
# 첫번째 bot 이미지를 기준 이미지로 설정
reference_bot = cv2.cvtColor(np.array(Image.open(Bot_path + 'Bot_0.jpg')), cv2.COLOR_RGB2BGR)
reference_bot_hsv = cv2.cvtColor(reference_bot, cv2.COLOR_BGR2HSV)

In [12]:
for i in range(len(img_path_list)):
    top = cv2.cvtColor(np.array(Image.open(Top_path + 'Top_{}.jpg'.format(i))), cv2.COLOR_RGB2BGR)
    bot = cv2.cvtColor(np.array(Image.open(Bot_path + 'Bot_{}.jpg'.format(i))), cv2.COLOR_RGB2BGR)
    # 첫 번째 bot 이미지와 각 bot 이미지 사이의 히스토그램 변환을 계산
    bot_hsv = cv2.cvtColor(bot, cv2.COLOR_BGR2HSV)
    bot_matched = hist_match(bot_hsv[:,:,2], reference_bot_hsv[:,:,2])
    # 계산한 히스토그램 변환을 top 이미지에 적용
    top_hsv = cv2.cvtColor(top, cv2.COLOR_BGR2HSV)
    top_hsv[:,:,2] = hist_match(top_hsv[:,:,2], bot_matched)
    top_matched = cv2.cvtColor(top_hsv, cv2.COLOR_HSV2BGR)
    # 보정된 top 이미지를 저장
    #Image.fromarray(cv2.cvtColor(top_matched, cv2.COLOR_BGR2RGB)).save(os.path.join(Top_path, 'Top_matched_{}.jpg'.format(i)))